In [1]:
import pandas as pd

In [45]:
data_df = pd.read_json("CMeEE_train.json")

In [46]:
dev_data_df = pd.read_json("CMeEE_dev.json")

In [47]:
data_df = data_df.append(dev_data_df, ignore_index=True)
data_df.shape

(20000, 2)

In [3]:
data_df = pd.read_json("CMeEE_test.json")

In [48]:
def _split_long_sent(query_data: list, max_seq_len=128, ratio_slide=1):
    """
    This function is for split the sentence which length over 128
    :param query_data: list of string
    :return: processed_query_data: list of string, length of each string smaller than 128
             concate_data_dict: map of raw text to splitted data
    """
    processed_query_data = []
    processed_seq_idx = []

    query_len = len(query_data)
    concate_data_dict = dict(zip(range(query_len), [[] for _ in range(query_len)]))

    for idx_sent, sent in enumerate(query_data):
        sent_len = len(str(sent))
        if sent_len > max_seq_len:  # parse the sentence if length larger than max len
            split_pos = 0
            while sent_len - split_pos > max_seq_len:  # if the rest part shorter than max seq_len, break
                is_split = False  # flag, if sentence does not contain any split symbol will be directly appended
                cur_head = split_pos + max_seq_len - 1

                alter_split_pos = None

                for idx, char in enumerate(sent[cur_head:split_pos:-1]):
                    if char in [",", ".", ";", "，", "。", "；"]:
                        processed_query_data.append(sent[split_pos:cur_head - idx + 1])
                        processed_seq_idx.append([split_pos, cur_head - idx])

                        split_pos += max_seq_len - idx
                        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)
                        is_split = True
                        break

                    if (not alter_split_pos) and char in [")", "）"]:
                        alter_split_pos = idx

                if not is_split:  # full sent added into list

                    if alter_split_pos is not None:
                        processed_query_data.append(sent[split_pos:cur_head - alter_split_pos + 1])
                        processed_seq_idx.append([split_pos, cur_head - alter_split_pos])

                        split_pos += max_seq_len - alter_split_pos
                        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)
                    else:
                        print(query_data)
                        processed_query_data.append(sent[split_pos:split_pos + max_seq_len])
                        processed_seq_idx.append([split_pos, split_pos + max_seq_len - 1])

                        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)
                        split_pos += max_seq_len

                slide_pos = int(split_pos - (1 - ratio_slide) * len(processed_query_data[-1]))
                for idx, char in enumerate(sent[slide_pos:split_pos]):
                    if char in [",", ".", ";", "，", "。", "；", ")", "）"]:
                        split_pos = idx + slide_pos + 1
                        break

            processed_query_data.append(sent[split_pos:])
            processed_seq_idx.append([split_pos, sent_len - 1])

        else:
            processed_query_data.append(sent)
            processed_seq_idx.append([0, sent_len - 1])

        concate_data_dict[idx_sent].append(len(processed_query_data) - 1)  # record the last span or short sentence

    return processed_query_data, concate_data_dict, processed_seq_idx

In [49]:
new_data = list()
for item in data_df.itertuples():
    region_dict = dict()
    entity_list = list()

    for entity in item.entities:
        start = entity["start_idx"]
        end = entity["end_idx"]
        region_dict[(start, end)] = {"type": entity["type"], "entity": entity["entity"]}

    seq_list, seq_dict, seq_idx_list = _split_long_sent([item.text], ratio_slide=0.5)

    for seq, seq_idx in zip(seq_list, seq_idx_list):
        seq_len = len(seq)
        sub_seq_region_list = list()
        for (start, end), entity in region_dict.items():
            if end <= seq_idx[1] and start >= seq_idx[0]:
                sub_seq_entity = {"start_idx": start - seq_idx[0], "end_idx": end - seq_idx[0], "type": entity["type"],
                                  "entity": entity["entity"]}
                sub_seq_region_list.append(sub_seq_entity)

                if entity["entity"] != seq[sub_seq_entity["start_idx"]:sub_seq_entity["end_idx"] + 1]:
                    print("error")

        new_data.append({"text": seq, "entities": sub_seq_region_list})


['相对来说比较常用的手术方法有尿道口前移龟头成形术（meata尿道口前移龟头成形术ndgmeataladvancementandglanuloplastyincorporatedprocedure或FMAGPIFlapMathieu，Filp-Flap尿道成形术la加盖岛状皮瓣尿道成形术成形onlayislandflap尿道成形术皮内板横行岛状皮瓣尿道成形术包皮内板横行岛状皮瓣尿道成形术十年Duckett术多的在无阴茎下弯的尿道下裂中应用尿道板纵切卷管尿道成形术（也尿道板纵切卷管尿道成形术。']
['参考文献1.陈清棠.临床神经病学.北京科学技术出版社，2000：12.李庆有，等.肝豆状核变性的脑MRI与临床表现的相关分析.中华神经科杂志，1997，30（6）：367-3673.吴斌.小儿肝豆状核变性的诊治新进展.国外医学儿科学分册，1997，24（4）：202-2024.杨任民.肝豆状核变性的治疗.中国实用儿科杂志，2000，15（2）：79-805.刘晓青，张雅芳，刘孜孜等.肝豆状核变性基因类型与临床表型关系研究.中华儿科杂志，2003，41（1）：35-356.LoudianosG，GitlinJD.Wilson’sdisease.SeminLiverDis，2000，20（3）：353-3537.RichardsRJ，HammittJK.Findingtheoptimalmultiple-teststrategyusingamethodanalogoustologisticregression：thediagnosisofhepatolenticulardegeneration（Wilson’sdisease）.MedDecisMaking，1996，16（4）：367-3678.RobertsEA，CoxDW.Wilsondisease.BaillieresClinGastroenterol，1998，12（2）：237-2379.SmoloarekC，StremmelW.TherapyofWilsondisease.ZGastroenterol，1999，37（4）：293-29310.中华医学会神经病学分会帕金森病及运动障碍学组.中华医学会神经病学分会神经遗传病学组.肝豆状核变性的诊断与治疗指南《中华神经科杂志》2008，41（8）：566-566'

In [50]:
# check
for item in pd.DataFrame(new_data).itertuples():
    for entity in item.entities:
        if item.text[entity["start_idx"]:entity["end_idx"] + 1] != entity["entity"]:
            print("error")

In [51]:
processed_data = []
for item in pd.DataFrame(new_data).itertuples():
    row_data_dict = dict()
    # insert space between every two chars
    row_data_dict["text_a"] = " ".join(list("".join((item.text.split()))))

    # generate sentence label
    label_sent = ["O"] * (len(item.text))
    region_dict = dict()

    for entity in item.entities:
        start = entity["start_idx"]
        end = entity["end_idx"]

        if start == end:
            label_sent[start] = "S"
            region_dict[(start, end)] = entity["type"]
            continue

        if label_sent[start] == "E" or label_sent[end] == "B":
            print(item, "boundary redandency")
        else:
            label_sent[start] = "B" if label_sent[start] != "S" else "S"
            label_sent[end] = "E" if label_sent[end] != "S" else "S"

            region_dict[(start, end)] = entity["type"]

            for ele_idx, elem_label in enumerate(label_sent[start + 1:end]):
                if elem_label == "O":
                    label_sent[ele_idx + start + 1] = "M"

    row_data_dict["bd_label"] = " ".join(label_sent)

    # generate region label
    label_region = []
    for start_idx, head in enumerate(label_sent):
        if head == "S" or head == "B":
            # single entity
            if head == "S":
                label_region.append(region_dict[(start_idx, start_idx)])

            # other entity
            for end_idx, tail in enumerate(label_sent[start_idx + 1:]):
                if tail == "S" or tail == "E":
                    tail_idx = start_idx + 1 + end_idx
                    label = region_dict[(start_idx, tail_idx)] if ((start_idx, tail_idx) in region_dict) else "O"
                    label_region.append(label)

                elif tail == "O":
                    break

    row_data_dict["entity_label"] = " ".join(label_region)
    if len(label_region):
        processed_data.append(row_data_dict)

Pandas(Index=10297, text='未用呼吸机的患儿可表现为三凹征，腹部肝脏肿大。', entities=[{'start_idx': 2, 'end_idx': 4, 'type': 'equ', 'entity': '呼吸机'}, {'start_idx': 12, 'end_idx': 14, 'type': 'dis', 'entity': '三凹征'}, {'start_idx': 16, 'end_idx': 16, 'type': 'bod', 'entity': '腹'}, {'start_idx': 14, 'end_idx': 17, 'type': 'sym', 'entity': '征，腹部'}]) boundary redandency


In [52]:
len(processed_data)

20900

In [53]:
processed_data_df = pd.DataFrame(data=processed_data, columns=["text_a", "bd_label", "entity_label"])

In [56]:
from sklearn.model_selection import train_test_split

In [64]:
train, test = train_test_split(processed_data_df, test_size=0.4)

In [65]:
train.shape

(12540, 3)

In [67]:
dev, test = train_test_split(test, test_size=0.5)

In [69]:
print(test.shape,dev.shape)

(4180, 3) (4180, 3)


In [70]:
train.to_csv("train.tsv", sep="\t", index=False)
dev.to_csv("dev.tsv", sep="\t", index=False)
test.to_csv("test.tsv", sep="\t", index=False)

In [14]:
processed_data_df.shape

(15652, 3)

In [21]:
processed_data_df.sample(32).to_csv("train.tsv", sep="\t", index=False)

In [38]:
15652 / 3

5217.333333333333